In [ ]:
from lstm import *
from mappings import *
from matplotlib import pyplot

import tensorflow as tf
import tensorflow.keras as k
import pandas as pdk
import glob

%load_ext autoreload
%autoreload 2

assert(tf.test.is_gpu_available()   and\
       tf.test.is_built_with_cuda() and\
       tf.test.is_built_with_gpu_support())


tf.__version__

In [ ]:
csv_files     = glob.glob('data/**/**.csv')
column_rename = {'country': 'area', 'countrycode': 'areacode'}
check_columns = ["note", "yearcode", "elementgroup"]

from mappings import get_mapping, get_area_mapping, extract_element_to_item

item_mapping    = get_mapping(csv_files, column_rename, check_columns, ['itemcode','item'])
element_mapping = get_mapping(csv_files, column_rename, check_columns, ['elementcode', 'element', 'unit'])
df              = pd.read_csv("df.csv")
element_to_item = extract_element_to_item(df)

In [ ]:
df.columns
print(df[~df['5510'].isna()].itemcode.value_counts())
#print([(e,element_mapping[int(e)]) for e,_ in element_to_item.items()])
print((7231,element_mapping[7231]))
print([(i,item_mapping[i]) for i in element_to_item[str(7231)]])

In [ ]:
#print([(e,element_mapping[int(e)]) for e,_ in element_to_item.items()])
print((5510,element_mapping[5510]))
print([(i,item_mapping[i]) for i in element_to_item[str(5510)]])

In [129]:
input_pairs   = [(5510,item_code) for item_code in (1765,1808,1738,1058,1062)]
output_pairs  = [(7231,1711)]
samples       = build_samples(df,20,input_pairs,output_pairs)

non_differentiated  = df[df.itemcode == 1711][['areacode','year',str(7231)]].set_index(['areacode','year'])
differentiated      = introduce_differentiation(non_differentialed)
adjustment_ratio          = ((differentiated.max()-differentiated.min())).values[0]
#avg_percentage_adjustment = differentiated.mean().values[0]

def adj_rms(y_true, y_pred): 
    return tf.sqrt(tf.reduce_mean(((y_true - y_pred)*(adjustment_ratio))**2))


train_X, train_Y, test_X, test_Y  = reshuffle_observations(samples,0.80,len(output_pairs))
print("Number of training_samples: ",len(train_X))


Number of training_samples:  4512


In [133]:
batch_size = 100
num_samples = train_X.shape[0]

model = build_lstm(num_input_timeseries=train_X[0].shape[1],
                   num_output_dimensions=train_Y[0].shape[1],
                   num_timesteps=train_X[0].shape[0],
                   model_complexity=80,
                   batch_size=batch_size )


model.compile(optimizer=k.optimizers.Adam(lr=0.0001),
              loss=k.losses.MSE,metrics=[adj_rms])

training_set = tf.data.Dataset.from_tensor_slices((    \
                            tf.cast(train_X, tf.float32), \
                            tf.cast(train_Y, tf.float32))).batch(batch_size).shuffle(num_samples)

testing_set = tf.data.Dataset.from_tensor_slices((
                            tf.cast(test_X,tf.float32),
                            tf.cast(test_Y,tf.float32))).batch(batch_size).shuffle(num_samples)
    
history      = model.fit(training_set,validation_data=testing_set,
                         epochs=1000,verbose=1, shuffle=False)

#pyplot.plot(history.history['loss'], label='train')
#pyplot.plot(history.history['loss'], label='test')
pyplot.plot(history.history['adj_rms'], label='train')
pyplot.plot(history.history['adj_rms'], label='test')
pyplot.legend()
pyplot.show()

Train on 46 steps, validate on 12 steps
Epoch 1/1000
46/46 [==============================] - 6s 129ms/step - loss: 0.0356 - adj_rms: 0.3318 - val_loss: 0.0297 - val_adj_rms: 0.3032
Epoch 2/1000
46/46 [==============================] - 1s 17ms/step - loss: 0.0172 - adj_rms: 0.2236 - val_loss: 0.0039 - val_adj_rms: 0.1079
Epoch 3/1000
46/46 [==============================] - 1s 17ms/step - loss: 0.0026 - adj_rms: 0.0875 - val_loss: 0.0028 - val_adj_rms: 0.0895
Epoch 4/1000
46/46 [==============================] - 1s 18ms/step - loss: 0.0025 - adj_rms: 0.0848 - val_loss: 0.0027 - val_adj_rms: 0.0886
Epoch 5/1000
46/46 [==============================] - 1s 19ms/step - loss: 0.0025 - adj_rms: 0.0848 - val_loss: 0.0027 - val_adj_rms: 0.0887
Epoch 6/1000
46/46 [==============================] - 1s 18ms/step - loss: 0.0025 - adj_rms: 0.0848 - val_loss: 0.0027 - val_adj_rms: 0.0887
Epoch 7/1000
46/46 [==============================] - 1s 17ms/step - loss: 0.0025 - adj_rms: 0.0848 - val_loss: 0

KeyboardInterrupt: 